### Import Required Packages

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Download Tiny-ImageNet: http://cs231n.stanford.edu/tiny-imagenet-200.zip

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

data_dir = 'tiny-imagenet-200'

train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## Define and Model

In [3]:
from Models.Vgg19 import VGG19

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG19(flatten_output = 8*8*256, num_classes=200).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(device,"\n", model)

cuda 
 VGG19(
  (convolution_layers): Sequential(
    (0): Convolution(
      (layer1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (convolutions): ModuleList(
        (0-1): 2 x Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Convolution(
      (layer1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (convolutions): ModuleList(
        (0-1): 2 x Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Convolution(
      (layer1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (convolutions): ModuleList(
        (0-1): 2 x Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Training & Validation

In [4]:
def validate(model, val_loader, device):
    model.eval()
    y_preds = []
    y_trues = []

    with torch.no_grad():
        for X, Y in val_loader:
            X = X.to(device)
            Y = Y.to(device)
            outputs = model(X)
            preds = torch.argmax(outputs, dim=1)
            y_preds.extend(preds.cpu().numpy())
            y_trues.extend(Y.cpu().numpy())

    acc = accuracy_score(y_trues, y_preds)
    print("Validation Accuracy:", acc)

In [5]:
NUM_EPOCHS = 1
iterations_per_epoch = len(train_loader)
for _ in range(20):
    model.train()
    total_loss = 0
    for i, (X, Y) in enumerate(train_loader):
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs,Y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if i % 50 == 0:
            print(f"[{_}][{i+1}/{iterations_per_epoch}]-----------------------------------------------------------[{loss.item()}]")
    validate(model, val_loader, device)

[0][1/3125]-----------------------------------------------------------[5.296179294586182]


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
